All HA and NA data analysed uses files filtered to only contain strains with the full motif. The dataframes with S/T data
therefore can contain 100 entries for some sites and 4000 for some sites, howevere when plotting relative circles it is only relative to usage at each site so those circles may appear similar in size when from different sites. Either be very clear about this or change it in future.

In [4]:
#import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import CommonMotifProb as MP
import matplotlib
from scipy.stats import chi2_contingency
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import random

In [7]:
#define files
path='Residue_Dataframes/'
#inputfiles=['H3_fixed_residue_Dataframe_non-motif-positions_allResidues.csv']
#inputfiles=['NP_fixed_residue_Dataframe_motif-positions_allResidues.csv']
#inputfiles=['NP_fixed_residue_Dataframe_non-motif-positions_allResidues.csv']
#inputfiles=['NS2_residues_Dataframe_motif-positions_allResidues.csv','M2_residues_Dataframe_motif-positions_allResidues.csv','PA_fixed_residue_Dataframe_motif-positions_allResidues.csv']#['NP_fixed_residue_Dataframe_motif-positions_allResidues.csv']
inputfiles=['NP_fixed_residue_Dataframe_motif-positions_allResidues.csv', 'M12_fixed_residue_Dataframe_motif-positions_allResidues.csv','NS12_residue_Dataframe_motif-positions_allResidues.csv','NS2_residues_Dataframe_motif-positions_allResidues.csv','M2_residues_Dataframe_motif-positions_allResidues.csv','PA_fixed_residue_Dataframe_motif-positions_allResidues.csv']
#inputfiles=['NP_fixed_residue_Dataframe_motif-positions_allResidues.csv', 'M12_fixed_residue_Dataframe_motif-positions_allResidues.csv','NS12_residue_Dataframe_motif-positions_allResidues.csv']#'A','B','C']
nonmotiffiles=['NP_fixed_residue_Dataframe_non-motif-positions_S_T.csv','M12_fixed_residue_Dataframe_non-motif-positions_S_T.csv','NS12_residue_Dataframe_non-motif-positions_S_T.csv']

In [8]:
#read in selected motifs file
#also read in motifs to compare with from full motifs file?
path='Residue_Dataframes/'
keyresiduedata=[pd.read_csv(path+i,index_col=0) for i in inputfiles]
keyresiduedataconcat=pd.concat(keyresiduedata)

nonmotifdata=[pd.read_csv(path+i,index_col=0) for i in nonmotiffiles]
nonmotifdata=pd.concat(nonmotifdata)

AAs=['S','T']


relevantcodons=[j.replace('U','T') for i in AAs for j in MP.AAtoCodonDICT[MP.SingleLetterDICT[i]]]


In [9]:
keyresiduedata=keyresiduedataconcat[keyresiduedataconcat['AminoAcid'].isin(AAs)]

In [10]:
columns = ['Site','Codon','Phospho']
nondatas=[]
sites=nonmotifdata.ResidueNo.unique()
for i in sites:
    temp=nonmotifdata[nonmotifdata['ResidueNo']==i]
    sitecodons=temp.Codon.value_counts()
    for j in sitecodons.keys():
        if sitecodons[j]>500:
            
            nondatas.append([i,j,0])

In [11]:
columns = ['Site','Codon','Phospho']
datas=[]
sites=keyresiduedata.ResidueNo.unique()
for i in sites:
    temp=keyresiduedata[keyresiduedata['ResidueNo']==i]
    sitecodons=temp.Codon.value_counts()
    for j in sitecodons.keys():
        if sitecodons[j]>500:
            datas.append([i,j,1])

In [16]:
#determining table assuming full or partial dependence
columns = ['Site','Codon','Phospho']
nondatas=[]
sites=nonmotifdata.ResidueNo.unique()
for i in sites:
    temp=nonmotifdata[nonmotifdata['ResidueNo']==i]
    sitecodons=temp.Codon.value_counts()
    for j in sitecodons.keys():
        if sitecodons[j]>1000:
            loopcount=sitecodons[j]/1000
            for l in range(int(loopcount)):
                nondatas.append([i,j,0])
            
            
columns = ['Site','Codon','Phospho']
datas=[]
sites=keyresiduedata.ResidueNo.unique()
for i in sites:
    temp=keyresiduedata[keyresiduedata['ResidueNo']==i]
    sitecodons=temp.Codon.value_counts()
    for j in sitecodons.keys():
        if sitecodons[j]>1000:
            loopcount=sitecodons[j]/1000
            for l in range(int(loopcount)):
                datas.append([i,j,1])

In [17]:

data=pd.DataFrame(datas,columns=columns)
nondata=pd.DataFrame(nondatas,columns=columns)

newdata=pd.concat([data,nondata])

In [18]:
newdata

,Site,Codon,Phospho
0,6,ACC,1
1,6,ACC,1
2,6,ACC,1
3,6,ACC,1
4,6,ACC,1
...,...,...,...
887,127,ACC,0
888,213,TCA,0
889,213,TCA,0
890,74,AGC,0


In [29]:
NPPhospho=keyresiduedata
NPPhospho['Phospho']=1
NPNonPho=nonmotifdata
NPNonPho['Phospho']=0

PhosphoConcat=pd.concat([NPPhospho,NPNonPho])

/var/folders/dt/9s3cdx090n1_ppdzkzk88tx40000gp/T/ipykernel_10290/3640571219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NPPhospho['Phospho']=1


In [21]:
data=PhosphoConcat

In [23]:
data_crosstab = pd.crosstab(newdata['Codon'], 
                            newdata['Phospho'],  
                               margins = False) 
print(data_crosstab) 

Phospho    0   1
Codon           
ACA      124  28
ACC       71  48
ACG       21  29
ACT      153  25
AGC       88   7
AGT      143   0
TCA      109  16
TCC       66  37
TCG       15  11
TCT      102  41


In [24]:
data_crosstab = pd.crosstab(data['Codon'], 
                            data['Phospho'],  
                               margins = False) 
print(data_crosstab) 

Phospho       0      1
Codon                 
ACA      148091  33601
ACC       90999  54095
ACG       31672  32429
ACT      173891  28829
AGC      107065   7218
AGT      161325    330
TCA      125139  18987
TCC       77635  41692
TCG       19723  12769
TCT      118200  47113


In [26]:
a=chi2_contingency(data_crosstab)
print (a)

Chi2ContingencyResult(statistic=152717.06651074922, pvalue=0.0, dof=9, expected_freq=array([[143865.11608405,  37826.88391595],
       [114886.53960053,  30207.46039947],
       [ 50755.66236325,  13345.33763675],
       [160515.24741077,  42204.75258923],
       [ 90490.15400476,  23792.84599524],
       [127999.66614142,  33655.33385858],
       [114120.06979245,  30005.93020755],
       [ 94484.03180636,  24842.96819364],
       [ 25727.41426041,   6764.58573959],
       [130896.098536  ,  34416.901464  ]]))


In [ ]:
nonmotifdata[0]
#dp=nonmotifdata[0]
#dp[dp['ResidueNo']==60]